## Introduction

The decision tree algorithm is a supervised learning algorithm -- we first construct the tree with historical data, and then use it to predict an outcome. One of the major advantages of decision trees is that they can pick up nonlinear interactions between variables in the data that linear regression can't. In our bear wrestling example, a decision tree can pick up on the fact that you should only wrestle large bears when escape is impossible, whereas a linear regression would have had to weight both factors in the absence of the other.

| Bear name  	| Size  	| Escape possible? 	| Action   	|
|------------	|-------	|------------------	|----------	|
| Yogi       	| Small 	| No               	| Wrestle  	|
| Winnie     	| Small 	| Yes              	| Wrestle  	|
| Baloo      	| Large 	| Yes              	| Run away 	|
| Gentle Ben 	| Large 	| No               	| Wrestle  	|

## Overview of the Data Set

We'll be looking at individual income in the United States. The data is from the 1994 census, and contains information on an individual's `marital status`, `age`, `type of work`, and more. The `target column`, or what we want to predict, is whether individuals make less than or `equal to 50k a year, or more than 50k a year`.

In [20]:
import pandas
import numpy as np

In [3]:
# Set index_col to False to avoid pandas thinking that the first column is row indexes (it's age)
income = pandas.read_csv("income.csv", index_col=False)
print(income.head(5))

   age          workclass  fnlwgt   education  education_num  \
0   39          State-gov   77516   Bachelors             13   
1   50   Self-emp-not-inc   83311   Bachelors             13   
2   38            Private  215646     HS-grad              9   
3   53            Private  234721        11th              7   
4   28            Private  338409   Bachelors             13   

        marital_status          occupation    relationship    race      sex  \
0        Never-married        Adm-clerical   Not-in-family   White     Male   
1   Married-civ-spouse     Exec-managerial         Husband   White     Male   
2             Divorced   Handlers-cleaners   Not-in-family   White     Male   
3   Married-civ-spouse   Handlers-cleaners         Husband   Black     Male   
4   Married-civ-spouse      Prof-specialty            Wife   Black   Female   

   capital_gain  capital_loss  hours_per_week  native_country high_income  
0          2174             0              40   United-States   

In [4]:
income.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             32561 non-null  int64 
 1   workclass       32561 non-null  object
 2   fnlwgt          32561 non-null  int64 
 3   education       32561 non-null  object
 4   education_num   32561 non-null  int64 
 5   marital_status  32561 non-null  object
 6   occupation      32561 non-null  object
 7   relationship    32561 non-null  object
 8   race            32561 non-null  object
 9   sex             32561 non-null  object
 10  capital_gain    32561 non-null  int64 
 11  capital_loss    32561 non-null  int64 
 12  hours_per_week  32561 non-null  int64 
 13  native_country  32561 non-null  object
 14  high_income     32561 non-null  object
dtypes: int64(6), object(9)
memory usage: 3.7+ MB


## Converting Categorical Variables

Before we get started with decision trees, we need to convert the categorical variables in our data set to numeric variables. This involves assigning a number to each category label, then converting all of the labels in a column to the corresponding numbers.

One strategy is to convert the columns to a categorical type. Under this approach, pandas will display the labels as strings, but internally store them as numbers so we can do computations with them. The numbers aren't always compatible with other libraries like Scikit-learn, though, so it's easier to just do the conversion to numeric upfront.

In [5]:
cat_columns = ['workclass', 'education', 'marital_status', 'occupation', 'relationship', 'race', 'sex', 'native_country', 'high_income']

for cat_column in cat_columns:
    col = pandas.Categorical(income[cat_column])
    income[cat_column] = col.codes

In [6]:
income.head()

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,high_income
0,39,7,77516,9,13,4,1,1,4,1,2174,0,40,39,0
1,50,6,83311,9,13,2,4,0,4,1,0,0,13,39,0
2,38,4,215646,11,9,0,6,1,4,1,0,0,40,39,0
3,53,4,234721,1,7,2,6,0,2,1,0,0,40,39,0
4,28,4,338409,9,13,2,10,5,2,0,0,0,40,5,0


## Splitting Data

![Jupyter](./DT-split.png)

In [9]:
income['workclass'].value_counts()

4    22696
6     2541
2     2093
0     1836
7     1298
5     1116
1      960
8       14
3        7
Name: workclass, dtype: int64

In [10]:
private_incomes = income[income['workclass'] == 4]

public_incomes = income[income['workclass'] != 4]

## Decision Trees as Flows of Data

When we performed the split, `9865` rows went to the left, where `workclass` does not equal `4`, and `22696` rows went to the right, where `workclass` equals `4`.

It's useful to think of a decision tree as a flow of rows of data. When we make a split, some rows will go to the right, and some will go to the left. As we build the tree deeper and deeper, each node will "receive" fewer and fewer rows.

Here's a look at the splits, and the number of rows that will exist at each node:

![Jupyter](./DT-data-flow.png)

## Splitting Data to Make Predictions

The nodes at the bottom of the tree, where we decide to stop splitting, are called terminal nodes, or leaves. When we do our splits, we aren't doing them randomly; we have an objective. Our goal is to ensure that we can make a prediction on future data. In order to do this, all rows in each leaf must have only one value for our target column.

We're trying to predict the `high_income` column.
If `high_income` is `1`, it means that the person has an income higher than 50k a year.
If `high_income` is `0`, it means that they have an income less than or equal to 50k a year.

After constructing a tree using the `income` data, we'll want to make predictions. In order to do this, we'll take a new row and feed it through our decision tree.

First, we check whether the person works in the private sector. If they do, we'll check to see whether they're native to the US, and so on.

Eventually, we'll reach a leaf. The leaf will tell us what value we should predict for `high_income`.

In order to be able to make this prediction, all of the rows in a leaf should only have a single value for `high_income`. This means that leaves can't have both `0` and `1` values in the `high_income` column. Each leaf can only have rows with the same values for our target column. If this isn't the case, we won't be able to make effective predictions.

We'll need to continue splitting nodes until we get to a point where all of the rows in a node have the same value for `high_income`.

## Overview of Data Set Entropy

Data scientists commonly use a metric called entropy for this purpose. Entropy refers to disorder. **The more "mixed together" 1s and 0s are, the higher the entropy. A data set consisting entirely of 1s in the high_income column would have low entropy**.

The formula for entropy looks like this:
    
$-\sum_{i=1}^{c} {\mathrm{P}(x_i) \log_b \mathrm{P}(x_i)}$


| age 	| high_income 	|
|-----	|-------------	|
| 25  	| 1           	|
| 50  	| 1           	|
| 30  	| 0           	|
| 50  	| 0           	|
| 80  	| 1           	|

We could compute its entropy like this:

$\begin{align}
-\sum_{i=1}^{c} {\mathrm{P}(x_i) \log_b \mathrm{P}(x_i)} \nonumber \\
-((2/5 * \log_2 2/5) + (3/5 * \log_2 3/5)) \nonumber \\ 
-(-0.5287712379549449 + -0.44217935649972373)\nonumber \\
.97 \nonumber \\
\end{align}$

We get less than one "bit" of information -- only .97 -- because there are slightly more 1s in the sample data than 0s. This means that if we were predicting a new value, we could guess that the answer is 1 and be right more often than wrong (because there's a .6 probability of the answer being 1). Due to this prior knowledge, we gain less than a full "bit" of information when we observe a new value.

## Overview of Data Set Entropy

Compute the entropy of the `high_income` column in the income dataframe, and assign the result to `income_entropy`.

In [18]:
income['high_income'].value_counts(normalize=True)

0    0.75919
1    0.24081
Name: high_income, dtype: float64

In [19]:
import math
# We'll do the same calculation we did above, but in Python
# Passing in 2 as the second parameter to math.log will take a base 2 log
possibilities = income['high_income'].value_counts(normalize=True).values
entropy = []

for p in possibilities:
    entropy.append(p*math.log(p, 2))

income_entropy = -1*sum(entropy)
print(income_entropy)

0.7963839552022132


## Information Gain

$IG(T,A) = Entropy(T)-\sum_{v\in A}\frac{|T_{v}|}{|T|} \cdot Entropy(T_{v})$

Compute the information gain for splitting on the `age` column of `income`.

* First, compute the median of `age`.
* Then, assign anything less than or equal to the median to the left branch, and anything greater than the median to the right branch.
* Compute the information gain and assign it to `age_information_gain`.

In [22]:
def calc_entropy(column):
    """
    Calculate entropy given a pandas series, list, or numpy array.
    """
    # Compute the counts of each unique value in the column
    counts = np.bincount(column)
    # Divide by the total column length to get a probability
    probabilities = counts / len(column)
    
    # Initialize the entropy to 0
    entropy = 0
    # Loop through the probabilities, and add each one to the total entropy
    for prob in probabilities:
        if prob > 0:
            entropy += prob * math.log(prob, 2)
    
    return -entropy

# Verify that our function matches our answer from earlier
entropy = calc_entropy([1,1,0,0,1])
print(entropy)

information_gain = entropy - ((.8 * calc_entropy([1,1,0,0])) + (.2 * calc_entropy([1])))
print(information_gain)

0.9709505944546686
0.17095059445466854


In [32]:
income_entropy = calc_entropy(income["high_income"])

In [33]:
median_age = income['age'].median()
median_age

37.0

In [34]:
left_split = income[income["age"] <= median_age]
right_split = income[income["age"] > median_age]

In [36]:
age_information_gain = income_entropy - \
                        ((left_split.shape[0] / income.shape[0]) * calc_entropy(left_split["high_income"]) + 
                        ((right_split.shape[0] / income.shape[0]) * calc_entropy(right_split["high_income"])))

In [37]:
age_information_gain

0.047028661304691965

## Finding the Best Split

In [38]:
def calc_information_gain(data, split_name, target_name):
    """
    Calculate information gain given a data set, column to split on, and target
    """
    # Calculate the original entropy
    original_entropy = calc_entropy(data[target_name])
    
    # Find the median of the column we're splitting
    column = data[split_name]
    median = column.median()
    
    # Make two subsets of the data, based on the median
    left_split = data[column <= median]
    right_split = data[column > median]
    
    # Loop through the splits and calculate the subset entropies
    to_subtract = 0
    for subset in [left_split, right_split]:
        prob = (subset.shape[0] / data.shape[0]) 
        to_subtract += prob * calc_entropy(subset[target_name])
    
    # Return information gain
    return original_entropy - to_subtract

In [39]:
# Verify that our answer is the same as on the last screen
print(calc_information_gain(income, "age", "high_income"))

columns = ["age", "workclass", "education_num", "marital_status", "occupation", "relationship", "race", "sex", "hours_per_week", "native_country"]

0.047028661304691965


In [43]:
information_gains = []
ig_dict = {}
highest_gain = ''
max_ig = -1

for col in columns:
    ig = calc_information_gain(income, col, 'high_income')
    if ig > max_ig:
        max_ig = ig
        highest_gain = col
    information_gains.append(ig)
    ig_dict[col] = ig

In [44]:
highest_gain

'marital_status'

In [45]:
information_gains

[0.047028661304691965,
 0.006810984054396618,
 0.06501298413277423,
 0.1114272573715438,
 0.0015822303843424645,
 0.04736241665026941,
 0.0,
 0.0,
 0.04062246867123487,
 0.00013457344495848567]

In [46]:
ig_dict

{'age': 0.047028661304691965,
 'workclass': 0.006810984054396618,
 'education_num': 0.06501298413277423,
 'marital_status': 0.1114272573715438,
 'occupation': 0.0015822303843424645,
 'relationship': 0.04736241665026941,
 'race': 0.0,
 'sex': 0.0,
 'hours_per_week': 0.04062246867123487,
 'native_country': 0.00013457344495848567}

## Build the Whole Tree

So far, we've been following the ID3 algorithm to construct decision trees. There are other algorithms like CART that use different measures for the split criterion. We'll learn more about these other algorithms in future missions.

In the next mission, we'll figure out how to recursively construct an entire tree using the ID3 algorithm.